# Análisis de los diferentes modelos

## 1. Importación de librerías y modulos

In [13]:
#%pip install -r requirements.txt
!pip install ragas

In [14]:
!pip install langchain-huggingface

In [15]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, answer_correctness
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_community.llms import Ollama
from langchain_chroma import Chroma
import tqdm
from module import *

In [16]:
emb_model = [
    "baai_small"
]

llm_model = [
    "llama3.1"
]

## 2. Carga de datos 

In [17]:
import os
os.getcwd()

'c:\\Users\\luisb\\Documents\\GitHub\\documentQuestionAnswering\\testing'

In [18]:
documents_loader = PyPDFDirectoryLoader("./data/test1/cat")
documents = documents_loader.load()

In [19]:
documents[0]

Document(metadata={'source': 'data\\test1\\cat\\constitucio_espanyola.pdf', 'page': 0}, page_content="  \n \n \nConstitució espanyola \n \n \nTEXT CONSOLIDAT \n \nVigència 17/02/2024 - \n \nLa Nació espanyola, amb el desig d'establir la justícia, la llibertat i la seguretat i de promoure el bé de \ntots els qui la integren, en ús de la seva sobirania, proclama la voluntat de:  \nGarantir la convivència democràtica dins la Constitució i les lleis de conformitat amb un ordre econòmic i \nsocial just.  \nConsolidar un Estat de Dret que asseguri l'imperi de la Llei com a expressió de la voluntat popular.  \nProtegir tots els espanyols i els pobles d'Espanya en l'exercici dels drets humans, les seves cultures i \ntradicions, llengües i institucions.  \nPromoure el progrés de la cultura i de l'economia per tal d'assegurar a tothom una qualitat de vida \ndigna.  \nEstablir una societat democràtica avançada, i  \nCol·laborar a l'enfortiment d'unes relacions pacífiques i de cooperació eficaç en

## 3. Creación de la base de datos vectorial

In [20]:
# Load the existing database.
db = Chroma(
    collection_name="test1",
    persist_directory="./database",
    embedding_function=get_embedding_function(emb_model[0])
)

In [21]:
# Adding documents with a progress bar

for document in documents:
    db.add_documents([document])
    print(".")


.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


In [22]:
results = db.similarity_search(query="La Nació espanyola, amb el desig d'establir la justícia, la llibertat i la seguretat",k=1)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

*   
 
 
Constitució espanyola 
 
 
TEXT CONSOLIDAT 
 
Vigència 17/02/2024 - 
 
La Nació espanyola, amb el desig d'establir la justícia, la llibertat i la seguretat i de promoure el bé de 
tots els qui la integren, en ús de la seva sobirania, proclama la voluntat de:  
Garantir la convivència democràtica dins la Constitució i les lleis de conformitat amb un ordre econòmic i 
social just.  
Consolidar un Estat de Dret que asseguri l'imperi de la Llei com a expressió de la voluntat popular.  
Protegir tots els espanyols i els pobles d'Espanya en l'exercici dels drets humans, les seves cultures i 
tradicions, llengües i institucions.  
Promoure el progrés de la cultura i de l'economia per tal d'assegurar a tothom una qualitat de vida 
digna.  
Establir una societat democràtica avançada, i  
Col·laborar a l'enfortiment d'unes relacions pacífiques i de cooperació eficaç entre tots els pobles de la 
Terra.  
En conseqüència, les Corts aproven i el poble espanyol ratifica la següent  
CONSTIT

## 4. Preguntas de interés

In [23]:
questions_docs_cat = [
    "Quins drets lingüístics reconeix la Constitució Espanyola?",
    "Què estableix la Constitució Espanyola sobre la detenció preventiva i els drets de les persones detingudes?",
    # "Quins són els valors superiors de l'ordenament jurídic espanyol segons la Constitució?",
    # "Quines condicions específiques es poden aplicar en un contracte indefinit per a persones amb discapacitat?",
    # "Com es defineix i s'estableix la durada del període de prova en un contracte indefinit segons el document?",
    # "Quins drets de bonificació a la Seguretat Social es mencionen per a contractes indefinits celebrats amb treballadors pertanyents al Sistema Nacional de Garantia Juvenil?",
]

# questions_docs_en = [
#     "What are the conditions under which an employment contract's probationary period may be extended or terminated?",
#     "What are the stipulations for holiday entitlement and the conditions under which holiday can be carried over to the next year?",
#     "What are the employer’s rights concerning termination of employment during and after the probationary period?",
#     "What are the proposed alternatives for the structure and election of the UK Parliament’s Second Chamber?",
#     "What rights are explicitly protected under the proposed Bill of Rights in the document?",
#     "How does the document propose constitutional amendments should be enacted or repealed?",
# ]

# questions_docs_es = [
#     "¿Qué derechos lingüísticos reconoce la Constitución Española?",
#     "¿Qué establece la Constitución Española sobre la detención preventiva y los derechos de las personas detenidas?",
#     "¿Cuáles son los valores superiores del ordenamiento jurídico español según la Constitución?",
#     "¿Qué condiciones específicas pueden aplicarse en un contrato indefinido para personas con discapacidad?",
#     "¿Cómo se define y establece la duración del período de prueba en un contrato indefinido según el documento?",
#     "¿Qué derechos de bonificación a la Seguridad Social se mencionan para contratos indefinidos celebrados con trabajadores pertenecientes al Sistema Nacional de Garantía Juvenil?"
# ]

questions = questions_docs_cat #+ questions_docs_en + questions_docs_es

In [24]:
questions

['Quins drets lingüístics reconeix la Constitució Espanyola?',
 'Què estableix la Constitució Espanyola sobre la detenció preventiva i els drets de les persones detingudes?']

In [25]:
contexts = []

for question in questions:
    # Get the top 5 most relevant documents
    results = db.similarity_search_with_score(question, k=5)

    # Make a list of the contexts
    question_contexts = []
    for doc, _score in results:
        question_contexts.append(doc.page_content)

    # Append the context sub-list to the list of contexts
    contexts.append(question_contexts)

In [26]:
contexts

[["  \n \n \nConstitució espanyola \n \n \nTEXT CONSOLIDAT \n \nVigència 17/02/2024 - \n \nLa Nació espanyola, amb el desig d'establir la justícia, la llibertat i la seguretat i de promoure el bé de \ntots els qui la integren, en ús de la seva sobirania, proclama la voluntat de:  \nGarantir la convivència democràtica dins la Constitució i les lleis de conformitat amb un ordre econòmic i \nsocial just.  \nConsolidar un Estat de Dret que asseguri l'imperi de la Llei com a expressió de la voluntat popular.  \nProtegir tots els espanyols i els pobles d'Espanya en l'exercici dels drets humans, les seves cultures i \ntradicions, llengües i institucions.  \nPromoure el progrés de la cultura i de l'economia per tal d'assegurar a tothom una qualitat de vida \ndigna.  \nEstablir una societat democràtica avançada, i  \nCol·laborar a l'enfortiment d'unes relacions pacífiques i de cooperació eficaç entre tots els pobles de la \nTerra.  \nEn conseqüència, les Corts aproven i el poble espanyol ratifi

In [27]:
from pprint import pprint
print(len(contexts))
print(len(contexts[0]))
print(len(question_contexts))

2
5
5


In [28]:
truths = []

for context in contexts:
    # Concatenate the contexts of each context
    context = " ".join(context)
    truths.append(context)

print(truths)

['  \n \n \nConstitució espanyola \n \n \nTEXT CONSOLIDAT \n \nVigència 17/02/2024 - \n \nLa Nació espanyola, amb el desig d\'establir la justícia, la llibertat i la seguretat i de promoure el bé de \ntots els qui la integren, en ús de la seva sobirania, proclama la voluntat de:  \nGarantir la convivència democràtica dins la Constitució i les lleis de conformitat amb un ordre econòmic i \nsocial just.  \nConsolidar un Estat de Dret que asseguri l\'imperi de la Llei com a expressió de la voluntat popular.  \nProtegir tots els espanyols i els pobles d\'Espanya en l\'exercici dels drets humans, les seves cultures i \ntradicions, llengües i institucions.  \nPromoure el progrés de la cultura i de l\'economia per tal d\'assegurar a tothom una qualitat de vida \ndigna.  \nEstablir una societat democràtica avançada, i  \nCol·laborar a l\'enfortiment d\'unes relacions pacífiques i de cooperació eficaç entre tots els pobles de la \nTerra.  \nEn conseqüència, les Corts aproven i el poble espanyol

In [29]:
len(truths)

2

Ahora creamos las respuestas esperadas para cada pregunta.

In [30]:
answers_docs_cat = [
    "La Constitució Espanyola reconeix el castellà com a llengua oficial de l'Estat i estableix que tots els espanyols tenen el deure de conèixer-la i el dret a usar-la. També reconeix que les altres llengües espanyoles seran oficials en les respectives Comunitats Autònomes d'acord amb els seus Estatuts, i protegeix la riquesa de les diferents modalitats lingüístiques d'Espanya com un patrimoni cultural que ha de ser objecte d'especial respecte i protecció.",
    "La Constitució Espanyola estableix que la detenció preventiva no pot durar més del temps estrictament necessari per a la realització de les averiguacions, i en tot cas, el detingut ha de ser posat en llibertat o a disposició de l'autoritat judicial en un termini màxim de 72 hores. A més, garanteix el dret a ser informat de forma immediata i comprensible sobre els drets i les raons de la detenció, així com el dret a l'assistència d'un advocat.",
    # "Segons la Constitució Espanyola, els valors superiors de l'ordenament jurídic espanyol són la llibertat, la justícia, la igualtat i el pluralisme polític.",
    # "En un contracte indefinit per a persones amb discapacitat, es poden aplicar condicions específiques com bonificacions en la quota empresarial a la Seguretat Social, subvencions, i adaptacions del lloc de treball segons les característiques de la persona. A més, es poden establir períodes de prova i d'adaptació al treball adequats a la situació del treballador.",
    # "La durada del període de prova en un contracte indefinit es defineix d'acord amb el que estableix l'Estatut dels Treballadors, respectant les normes legals vigents. Pot variar segons el grup professional o el nivell del treballador, i en alguns casos pot ser de fins a un any si s'acull a certes disposicions legals.",
    # "Per als contractes indefinits celebrats amb treballadors pertanyents al Sistema Nacional de Garantia Juvenil, es mencionen bonificacions de 300 euros mensuals en la cotització empresarial a la Seguretat Social durant un període de 6 mesos. En el cas de contractes a temps parcial, la bonificació s'ajusta proporcionalment al percentatge de la jornada.",
]

# answers_docs_en = [
#     "The probationary period may be extended or terminated if the employee's work performance is not up to the required standard or if the working relationship is not agreeable. Both parties are required to give one week's written notice during this period.",
#     "The holiday entitlement is 5.6 weeks per year, pro-rata for part-time employees. Holidays must be taken within the year, and carrying over untaken holidays requires prior agreement, except in cases of sickness preventing the leave.",
#     "During the probationary period, both parties must give one week's notice to terminate employment. After the probationary period, four weeks' notice is required, increasing by one week per year of service up to a maximum of twelve weeks. The employer may terminate employment without notice in cases of serious breach, gross misconduct, or gross negligence.",
#     "The document offers alternatives for the Second Chamber: a directly elected body representing the nations and regions of the UK, or a Chamber of Experts appointed for 15 years based on recognized expertise. It also suggests the possibility of the Second Chamber having the power to veto bills under certain conditions.",
#     "The proposed Bill of Rights includes rights such as the right to life, freedom from torture, slavery, and forced labor, and the right to a fair trial. Other rights include freedom of thought, expression, assembly, and the right to marry and found a family.",
#     "The document proposes that constitutional amendments could be enacted or repealed by Parliament through ordinary legislative procedures, with some suggesting the need for a two-thirds majority in both Houses and approval by a public referendum."
# ]

# answers_docs_es = [
#     "La Constitución Española reconoce el castellano como lengua oficial del Estado y establece que todos los españoles tienen el deber de conocerla y el derecho a usarla. También reconoce que las demás lenguas españolas serán oficiales en las respectivas Comunidades Autónomas de acuerdo con sus Estatutos, y protege la riqueza de las distintas modalidades lingüísticas de España como un patrimonio cultural que debe ser objeto de especial respeto y protección.",
#     "La Constitución Española establece que la detención preventiva no puede durar más del tiempo estrictamente necesario para la realización de las averiguaciones, y en todo caso, el detenido debe ser puesto en libertad o a disposición de la autoridad judicial en un plazo máximo de 72 horas. Además, garantiza el derecho a ser informado de forma inmediata y comprensible sobre los derechos y las razones de la detención, así como el derecho a la asistencia de un abogado.",
#     "Según la Constitución Española, los valores superiores del ordenamiento jurídico español son la libertad, la justicia, la igualdad y el pluralismo político.",
#     "En un contrato indefinido para personas con discapacidad, se pueden aplicar condiciones específicas como bonificaciones en la cuota empresarial a la Seguridad Social, subvenciones, y adaptaciones del puesto de trabajo según las características de la persona. Además, se pueden establecer períodos de prueba y de adaptación al trabajo adecuados a la situación del trabajador.",
#     "La duración del período de prueba en un contrato indefinido se define de acuerdo con lo establecido en el Estatuto de los Trabajadores, respetando las normas legales vigentes. Puede variar según el grupo profesional o el nivel del trabajador, y en algunos casos puede ser de hasta un año si se acoge a ciertas disposiciones legales.",
#     "Para los contratos indefinidos celebrados con trabajadores pertenecientes al Sistema Nacional de Garantía Juvenil, se mencionan bonificaciones de 300 euros mensuales en la cotización empresarial a la Seguridad Social durante un período de 6 meses. En el caso de contratos a tiempo parcial, la bonificación se ajusta proporcionalmente al porcentaje de la jornada.",
# ]


answers = answers_docs_cat #+ answers_docs_en + answers_docs_es

Finalmente creamos el DataFrame para evaluar los modelos.

In [31]:
len(questions)

2

In [32]:
len(answers)

2

In [33]:
len(contexts)

2

In [ ]:
articles_tecnologia_cat = [
    "La intel·ligència artificial està transformant ràpidament la forma en què les empreses operen, automatitzant processos i millorant la presa de decisions a través de l'anàlisi de dades en temps real.",
    answers[1],
    # "El desenvolupament de la tecnologia 5G permetrà velocitats de connexió a internet significativament més altes, obrint la porta a noves aplicacions com la realitat augmentada i la Internet de les Coses (IoT).",
    # "Els vehicles elèctrics estan guanyant popularitat a mesura que els consumidors busquen alternatives més ecològiques als motors de combustió interna. A més, els avenços en bateries estan millorant l'autonomia i el temps de càrrega.",

    # "Les energies renovables, com l'energia solar i eòlica, estan experimentant un creixement ràpid gràcies a la seva disminució de costos i a la consciència creixent sobre el canvi climàtic i la sostenibilitat.",
    # "La computació en núvol s'ha convertit en una eina essencial per a les empreses, permetent emmagatzematge flexible i accés remot a les dades, així com la capacitat de treballar col·laborativament des de qualsevol lloc del món.",
]
articles_tecnologia_cat

["La intel·ligència artificial està transformant ràpidament la forma en què les empreses operen, automatitzant processos i millorant la presa de decisions a través de l'anàlisi de dades en temps real.",
 "La Constitució Espanyola estableix que la detenció preventiva no pot durar més del temps estrictament necessari per a la realització de les averiguacions, i en tot cas, el detingut ha de ser posat en llibertat o a disposició de l'autoritat judicial en un termini màxim de 72 hores. A més, garanteix el dret a ser informat de forma immediata i comprensible sobre els drets i les raons de la detenció, així com el dret a l'assistència d'un advocat."]

In [34]:
data_samples = {
    'question': questions,
    'answer': answers,
    'contexts': contexts,
    'ground_truth':  answers
}

dataset = Dataset.from_dict(data_samples)

## 5. Evaluación de los modelos

In [35]:
embeddings = get_embedding_function(emb_model[0])
llm = Ollama(model=llm_model[0])

In [36]:
import getpass
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass(
    "Enter your Hugging Face API key: "
)

In [37]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")


from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

end = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.3", max_new_tokens=512)

huggingface_llm = ChatHuggingFace(llm=end, tokenizer=tokenizer)
huggingface_embeddings =embeddings


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3.
401 Client Error. (Request ID: Root=1-66d06af2-63cb18360b1a37101eadc253;c5581b56-c053-44ce-936a-b935ecfbb01b)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3/resolve/main/config.json.
Access to model mistralai/Mistral-7B-Instruct-v0.3 is restricted. You must be authenticated to access it.

In [42]:
import nest_asyncio
from ragas.run_config import RunConfig


nest_asyncio.apply()

try:
    # score = evaluate(
    #     dataset,
    #     # metrics=[context_precision],
    #     metrics=[answer_correctness],
    #     llm=llm,
    #     embeddings=embeddings,
    #     raise_exceptions=False,
    #     run_config=RunConfig(
    #         max_retries=10, # Default is 10
    #         max_wait=160, # Default is 60
    #         max_workers=64 # Default is 16
    #     )
    # )


    score = evaluate(
        dataset=dataset,
        metrics=[answer_correctness],
        llm=llm,
        embeddings=embeddings,
        raise_exceptions=False
    )

except Exception as e:
    print(f"An error ocurred: {e}")

df_score = score.to_pandas()
print(df_score)

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Exception raised in Job[1]: OllamaEndpointNotFoundError(Ollama call failed with status code 404.)
Exception raised in Job[0]: OllamaEndpointNotFoundError(Ollama call failed with status code 404.)


                                            question  \
0  Quins drets lingüístics reconeix la Constituci...   
1  Què estableix la Constitució Espanyola sobre l...   

                                              answer  \
0  La Constitució Espanyola reconeix el castellà ...   
1  La Constitució Espanyola estableix que la dete...   

                                            contexts  \
0  [  \n \n \nConstitució espanyola \n \n \nTEXT ...   
1  [  \n \n \nConstitució espanyola \n \n \nTEXT ...   

                                        ground_truth  answer_correctness  
0  La Constitució Espanyola reconeix el castellà ...                 NaN  
1  La Constitució Espanyola estableix que la dete...                 NaN  


In [43]:
df_score.head(10)

,question,answer,contexts,ground_truth,answer_correctness
0,Quins drets lingüístics reconeix la Constituci...,La Constitució Espanyola reconeix el castellà ...,[ \n \n \nConstitució espanyola \n \n \nTEXT ...,La Constitució Espanyola reconeix el castellà ...,NaN
1,Què estableix la Constitució Espanyola sobre l...,La Constitució Espanyola estableix que la dete...,[ \n \n \nConstitució espanyola \n \n \nTEXT ...,La Constitució Espanyola estableix que la dete...,NaN


In [44]:
from datasets import load_dataset

# loading the V2 dataset
amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")
amnesty_qa

Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    eval: Dataset({
        features: ['question', 'ground_truth', 'answer', 'contexts'],
        num_rows: 20
    })
})

In [45]:
dir(amnesty_qa)

['__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_values_features',
 '_check_values_type',
 'align_labels_with_mapping',
 'cache_files',
 'cast',
 'cast_column',
 'class_encode_column',
 'cleanup_cache_files',
 'clear',
 'column_names',
 'copy',
 'data',
 'filter',
 'flatten',
 'flatten_indices',
 'formatted_as',
 'from_csv',
 'from_json',
 'from_parquet',
 'from_text',
 'fromkeys',
 'get',
 'items',
 'keys',
 'load_from_disk',
 'map',
 'num_columns',
 'num_r

In [46]:
amnesty_qa.column_names

{'eval': ['question', 'ground_truth', 'answer', 'contexts']}

In [47]:
dir(amnesty_qa["eval"])

['_TF_DATASET_REFS',
 '__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getitems__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_local_temp_path',
 '_check_index_is_initialized',
 '_data',
 '_estimate_nbytes',
 '_fingerprint',
 '_format_columns',
 '_format_kwargs',
 '_format_type',
 '_generate_tables_from_cache_file',
 '_generate_tables_from_shards',
 '_get_cache_file_path',
 '_get_output_signature',
 '_getitem',
 '_indexes',
 '_indices',
 '_info',
 '_map_single',
 '_new_dataset_with_indices',
 '_output_all_columns',
 '_push_parquet_shards_to_hub',
 '_save_to_disk_single',


In [48]:
df = amnesty_qa["eval"].to_pandas()
df

,question,ground_truth,answer,contexts
0,What are the global implications of the USA Su...,The global implications of the USA Supreme Cou...,The global implications of the USA Supreme Cou...,"[- In 2022, the USA Supreme Court handed down ..."
1,Which companies are the main contributors to G...,"According to the Carbon Majors database, the m...","According to the Carbon Majors database, the m...","[In recent years, there has been increasing pr..."
2,Which private companies in the Americas are th...,The largest private companies in the Americas ...,"According to the Carbon Majors database, the l...",[The issue of greenhouse gas emissions has bec...
3,What action did Amnesty International urge its...,Amnesty International urged its supporters to ...,Amnesty International urged its supporters to ...,"[In the case of the Ogoni 9, Amnesty Internati..."
4,What are the recommendations made by Amnesty I...,The recommendations made by Amnesty Internatio...,Amnesty International made several recommendat...,"[In recent years, Amnesty International has fo..."
5,Who are the target audience of the two books c...,The target audience of the two books created b...,The target audience of the two books created b...,"[In addition to children, parents, teachers, a..."
6,Which right guarantees access to comprehensive...,The right that guarantees access to comprehens...,The right that guarantees access to comprehens...,[The right to truth is a fundamental human rig...
7,Who has the right to be fully informed about h...,The victims of gross human rights violations a...,Everyone has the right to be fully informed ab...,"[In many cases, the identities of perpetrators..."
8,When can individuals be found guilty under Art...,Individuals can be found guilty under Article ...,Under Article 207.3 of the Russian Criminal Co...,[Article 207.3 of the Russian Criminal Code pe...
9,When does the prosecution consider statements ...,The prosecution considers statements contrary ...,Under Article 207.3 of the Russian Criminal Co...,[- As long as their statements are contrary to...


In [49]:
df["contexts"].iloc[0]

array(["- In 2022, the USA Supreme Court handed down a decision ruling that overturned 50 years of jurisprudence recognizing a constitutional right to abortion.\n- This decision has had a massive impact: one in three women and girls of reproductive age now live in states where abortion access is either totally or near-totally inaccessible.\n- The states with the most restrictive abortion laws have the weakest maternal health support, higher maternal death rates, and higher child poverty rates.\n- The USA Supreme Court ruling has also had impacts beyond national borders due to the geopolitical and cultural influence wielded by the USA globally and the aid it funds.\n- SRR organizations and activists across the world have expressed fear about the ruling laying the groundwork for anti-abortion legislative and policy attacks in other countries.\n- Advocates have also observed the ruling's impact on progressive law reform and the stalling of the adoption and enforcement of abortion guidelin

In [51]:
df["contexts"].iloc[5]

array(["In addition to children, parents, teachers, and caregivers are also key target audiences for Amnesty International's books on child rights. These adults play a crucial role in guiding and supporting children as they learn about their rights and responsibilities. By reading and discussing these books together, families and educators can help instill values of equality, respect, and compassion in the next generation.",
       "Amnesty International's books on child rights are designed to educate and empower young readers about their rights and the importance of standing up for justice. The colorful illustrations and engaging storytelling make these books accessible to children of various ages and backgrounds, sparking their curiosity and empathy for others.",
       'Amnesty International has therefore created two simple books on child rights and how to claim them for self and others: (1) ‘Know Your Rights and Claim Them: A Guide for Youth’, by Amnesty International, Angelina Jol

In [52]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 2
})

In [53]:
dataset.to_pandas()

,question,answer,contexts,ground_truth
0,Quins drets lingüístics reconeix la Constituci...,La Constitució Espanyola reconeix el castellà ...,[ \n \n \nConstitució espanyola \n \n \nTEXT ...,La Constitució Espanyola reconeix el castellà ...
1,Què estableix la Constitució Espanyola sobre l...,La Constitució Espanyola estableix que la dete...,[ \n \n \nConstitució espanyola \n \n \nTEXT ...,La Constitució Espanyola estableix que la dete...


In [54]:
df2 = dataset.to_pandas()
df2["contexts"]

0    [  \n \n \nConstitució espanyola \n \n \nTEXT ...
1    [  \n \n \nConstitució espanyola \n \n \nTEXT ...
Name: contexts, dtype: object

In [55]:
df2["contexts"].iloc[0]

array(["  \n \n \nConstitució espanyola \n \n \nTEXT CONSOLIDAT \n \nVigència 17/02/2024 - \n \nLa Nació espanyola, amb el desig d'establir la justícia, la llibertat i la seguretat i de promoure el bé de \ntots els qui la integren, en ús de la seva sobirania, proclama la voluntat de:  \nGarantir la convivència democràtica dins la Constitució i les lleis de conformitat amb un ordre econòmic i \nsocial just.  \nConsolidar un Estat de Dret que asseguri l'imperi de la Llei com a expressió de la voluntat popular.  \nProtegir tots els espanyols i els pobles d'Espanya en l'exercici dels drets humans, les seves cultures i \ntradicions, llengües i institucions.  \nPromoure el progrés de la cultura i de l'economia per tal d'assegurar a tothom una qualitat de vida \ndigna.  \nEstablir una societat democràtica avançada, i  \nCol·laborar a l'enfortiment d'unes relacions pacífiques i de cooperació eficaç entre tots els pobles de la \nTerra.  \nEn conseqüència, les Corts aproven i el poble espanyol r